In [1]:
import pandas as pd
import dask.dataframe as dd

import numpy as np
from datetime import datetime

from pathlib import Path
import glob  

In [2]:
# loading metadata file
filePath = '/'
df = dd.read_csv(filePath + '.csv').drop_duplicates()

# Pre-processing to filter out inconsistent data
df = df[df['Number of Occupants'] < df['Number of Occupants'].quantile(.999)]
df = df[df['Number of Floors'] < df['Number of Floors'].quantile(.999)]
df = df[df['Floor Area [ft2]'] < df['Floor Area [ft2]'].quantile(.999)]
df = df[df['Number of Remote Sensors'] < df['Number of Remote Sensors'].quantile(.999)]

In [3]:
lista = []
# List all files in directory using pathlib
basepath = Path("/")
files_in_basepath = (entry for entry in basepath.iterdir() if entry.is_file())
for item in files_in_basepath:
    lista.append(item.name)
lista_FileName = [s.replace('.csv', '') for s in lista]  # to have only the name of the file

In [4]:
# Create dataset with only Identifiers in 2019 
df_in_2019 = df[df.Identifier.isin(lista_FileName)]

In [5]:
# Creation of datasets for 1 and 2 occupants
df1 = df_in_2019[(df_in_2019['Number of Occupants'] == 1 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]
df2 = df_in_2019[(df_in_2019['Number of Occupants'] == 4 ) & (df_in_2019['Number of Remote Sensors'] >= 3)] # dataframe with the data I want to save

In [10]:
# Take all Identifiers of df2, to after open them
lista1=df2.loc[:,'Identifier'].compute().tolist()
lista=[]
for i in lista1:
    lista.append(mainPath+i+'.csv')

In [11]:
df_concat=pd.DataFrame()

for Idi in lista1:
     
    df_lista = pd.read_csv(mainPath + Idi+ '.csv', usecols = use_cols)

    # perform data filtering 
    df_lista_2 = df_lista.loc[:, df_lista.columns.str.contains('_Motion')].fillna(0)
    df_lista_2['DateTime']= df_lista['DateTime']
    
    df_lista_2['DateTime'] = pd.to_datetime(df_lista_2['DateTime'])
    df_lista_2 = df_lista_2.set_index(['DateTime'])
    
    #sum motion
    df_lista_2['activation'] = df_lista_2.loc[:, df_lista_2.columns.str.contains('_Motion')].sum(axis=1)
    # rolling
    df_resamble = pd.DataFrame(df_lista_2['activation'].rolling('10T').sum())
    # shift
    df_resamble['activation']=np.roll(df_resamble['activation'],-1)
    # and logical
    df_and = df_resamble*pd.DataFrame(df_lista_2['activation'] )
    df_greater_1 = df_lista_2[df_and> 2].fillna(0)
     
    final_df = df_greater_1.drop(df_greater_1.loc[:, df_greater_1.columns.str.contains('_Motion')], axis=1)
    final_df = pd.DataFrame(final_df.to_records())
    final_df['path']= Idi # Add the identifier to know to what data it corresponds
    final_df.to_csv('/'+Idi+ '.csv') # save the file

In [12]:
ff=dd.read_csv('/*', parse_dates=['DateTime']).drop('Unnamed: 0',axis=1)

In [13]:
ff['dayofweek']=ff['DateTime'].dt.day_name()
ff['Time'] = ff['DateTime'].dt.time
ff['Month'] = ff['DateTime'].dt.month


In [14]:
ff_columns = ['path', 'activation', 'dayofweek', 'Time', 'Month']
result = dd.merge(df2[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result['mean_activation']=(result['activation']/(result['Number of Remote Sensors']))  
result_group=result[['Identifier','dayofweek','Time','mean_activation']].groupby(['Identifier','Time','dayofweek']).mean()
df_result_group=dd.from_array(result_group.to_records()) 

In [15]:
df_result_group["TimeSTR"]=df_result_group["Time"].apply((lambda x: x.strftime('%H:%M')), meta='str')
copy_df_result_group = df_result_group.copy()
new = copy_df_result_group["TimeSTR"]
# concatenating Time with dayofweek column 
# overwriting name column 
copy_df_result_group["dayofweek"]= copy_df_result_group["dayofweek"].str.cat(new, sep =", ") 

In [16]:
# hace el mean de los valores de activation que coinciden para una misma posicion en la matriz
df_bb=copy_df_result_group.drop(['Time'],axis=1)
df_bb = df_bb.categorize(columns='dayofweek')
table = dd.pivot_table(df_bb, values='mean_activation', index='Identifier',
                    columns='dayofweek')

In [17]:
t=dd.from_array(table.to_records())
result_f = dd.merge(df2,t, on='Identifier').drop(['filename'],axis=1)

In [18]:
(result_f.compute()).to_csv('.csv')